# Exercise 01: Sampling and Aliasing

This exercise will guide you
through the process of sampling a continuous-time waveform
and the phenomenon of aliasing.

**This exercise is a doosey**,
so find a quiet place
and bev-up before you begin.

And remember: everything here *is* within your grasp!  Don't panic!

Let's install some packages:

In [ ]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -U scikit-learn numpy matplotlib ipywidgets ipympl

and import some stuff:

In [ ]:
%matplotlib widget

import ipywidgets as widgets
import numpy as np
import scipy as sp
import sklearn
import matplotlib.pyplot as plt

The following code creates a plot widget below.  Feel free to dig into it if you want.

In [ ]:
def create_plot():
    
    def generate_data (tone_freq_in__Hz, sample_rate__Hz):
        res = sklearn.utils.Bunch()
        
        res.tone_freq_in__Hz = tone_freq_in__Hz
        res.sample_rate__Hz  = sample_rate__Hz
        
        tone_freq_out__Hz = res.tone_freq_in__Hz
        while tone_freq_out__Hz <= -sample_rate__Hz/2:
            tone_freq_out__Hz += sample_rate__Hz
        while tone_freq_out__Hz > +sample_rate__Hz/2:
            tone_freq_out__Hz -= sample_rate__Hz

        res.tone_freq_out__Hz = tone_freq_out__Hz
        
        res.sample_period__sec = 1.0/res.sample_rate__Hz;
    
        res.t_lo__sec =  0.0;
        res.t_hi__sec =  1.0;
        
        res.ts_tone__sec = np.linspace(res.t_lo__sec, res.t_hi__sec, 1000)
        res.ts_samp__sec = np.arange(res.t_lo__sec, res.t_hi__sec, res.sample_period__sec); 

        res.ys_sin_in   = np.sin(2*np.pi * res.tone_freq_in__Hz  * res.ts_tone__sec)
        res.ys_sin_samp = np.sin(2*np.pi * res.tone_freq_in__Hz  * res.ts_samp__sec)
        res.ys_sin_out  = np.sin(2*np.pi * res.tone_freq_out__Hz * res.ts_tone__sec)
    
        res.ys_cos_in   = np.cos(2*np.pi * res.tone_freq_in__Hz  * res.ts_tone__sec)
        res.ys_cos_samp = np.cos(2*np.pi * res.tone_freq_in__Hz  * res.ts_samp__sec)
        res.ys_cos_out  = np.cos(2*np.pi * res.tone_freq_out__Hz * res.ts_tone__sec)
    
        res.N_fft        = res.ts_samp__sec.size
        res.bin_size__Hz = res.sample_rate__Hz/res.N_fft;
        
        res.fs_fft = np.linspace(-res.sample_rate__Hz/2, +res.sample_rate__Hz/2-res.bin_size__Hz, res.N_fft) 
    
        res.ys_sin_fft = sp.fft.fftshift(np.abs(np.fft.fft(res.ys_sin_samp, res.N_fft))/res.N_fft)
        res.ys_cos_fft = sp.fft.fftshift(np.abs(np.fft.fft(res.ys_cos_samp, res.N_fft))/res.N_fft)

        return res

    slider_tone_freq__Hz   = widgets.FloatSlider(min=-50.0, max= +50.0, step=0.1, value= 10.0, description="tone freq [Hz]",   continuous_update=True)
    slider_sample_rate__Hz = widgets.FloatSlider(min=  4.0, max=+100.0, step=0.1, value= 50.0, description="sample rate [Hz]", continuous_update=True)

    rb_waveform = widgets.RadioButtons(
        options=['sin', 'cos'],
        value='cos',
        description='plotted waveform',
        disabled=False
    )

    fig = plt.figure(figsize=(12,5))
    fig.canvas.toolbar_visible = False
    fig.canvas.resizable       = False
    
    ax_t = fig.add_subplot(1, 2, 1)
    ax_f = fig.add_subplot(1, 2, 2)

    def update_lines(dummy):
        res = generate_data(slider_tone_freq__Hz.value, slider_sample_rate__Hz.value)
        
        ax_t.clear()
        ax_f.clear()

        ax_t.set_ylim(res.t_lo__sec, res.t_hi__sec)
        ax_t.set_ylim(-1.25, +1.75)
        plt.sca(ax_t)
        plt.xlabel(r'time [sec]')
        plt.grid(visible=True, which='major', axis='y')

        sin_is_visible = rb_waveform.index == 0
        
        sin_in_line,   = ax_t.plot(res.ts_tone__sec, res.ys_sin_in,   color='r', linewidth=5.0, visible=sin_is_visible)
        sin_samp_line, = ax_t.plot(res.ts_samp__sec, res.ys_sin_samp, color='b', linewidth=0.0, visible=sin_is_visible, marker='o')
        sin_out_line,  = ax_t.plot(res.ts_tone__sec, res.ys_sin_out,  color='k', linewidth=3.0, visible=sin_is_visible)
    
        cos_in_line,   = ax_t.plot(res.ts_tone__sec, res.ys_cos_in,   color='r', linewidth=5.0, visible=not sin_is_visible)
        cos_samp_line, = ax_t.plot(res.ts_samp__sec, res.ys_cos_samp, color='b', linewidth=0.0, visible=not sin_is_visible, marker='o')
        cos_out_line,  = ax_t.plot(res.ts_tone__sec, res.ys_cos_out,  color='k', linewidth=3.0, visible=not sin_is_visible)

        if sin_is_visible:
            ax_t.legend([sin_in_line, sin_out_line, sin_samp_line], ['sin in', 'sin out', 'samples'])
        else:
            ax_t.legend([cos_in_line, cos_out_line, cos_samp_line], ['cos in', 'cos out', 'samples'])
    
        line_sin_fft, = ax_f.plot(res.fs_fft, res.ys_sin_fft,  color='b', linewidth=1.0, visible=sin_is_visible)
        line_cos_fft, = ax_f.plot(res.fs_fft, res.ys_cos_fft,  color='b', linewidth=1.0, visible=not sin_is_visible)

        ax_f.set_xlim(res.fs_fft[0], res.fs_fft[-1])
        ax_f.set_ylim(0, +1.25)
        plt.sca(ax_f)
        plt.xlabel(r'freq [Hz]')
        plt.grid(visible=True, which='major', axis='y')

        if sin_is_visible:
            ax_f.legend([line_sin_fft], ['|mag|'])
        else:
            ax_f.legend([line_cos_fft], ['|mag|'])

        fig.canvas.draw_idle()

    update_lines('dummy')
        
    slider_tone_freq__Hz.observe(update_lines, names='value')
    slider_sample_rate__Hz.observe(update_lines, names='value')
    rb_waveform.observe(update_lines, names='value')
    
    display(
        widgets.HBox([
            rb_waveform,
            widgets.VBox(
                [slider_tone_freq__Hz, slider_sample_rate__Hz]
            )
        ])
    )
    
create_plot()

Note the figure number above the plots.  If it starts getting large, you _might_ want to run this command
to close all open plots and free up memory:

In [ ]:
plt.close('all')

Let's get familiar with the plots.

The left plot shows time-domain data.
The input waveform (red) will be either a **sine** or a **cosine** at the **tone freq**,
and will be sampled (blue) at the **sample rate**.
The output waveform after sampling will be in black.

The right plot shows frequency-domain data.

Remember that you can type values into the sliders if you want.
(Make sure you **press return** and NOT shift-return when doing so!)

Make sure you are looking at the default settings.

**What is the duration of the displayed time waveform?**

YOUR ANSWER HERE

**What is the period of the displayed time waveform?**

YOUR ANSWER HERE

**Compute the frequency (in Hz) of the displayed time waveform from its period (show your work).  Is it what you expected?**

YOUR ANSWER HERE

**Compute the frequency (in rad/sec) of the displayed time waveform from its frequency in Hz (show your work).**

YOUR ANSWER HERE

**How do you know this waveform is a cosine and not a sine?**

YOUR ANSWER HERE

**MAKE SURE YOU CAN ANSWER THE ABOVE QUESTIONS BEFORE CONTINUING!**

Toggle to the sine waveform.

**Repeat the verification steps above for the sine waveform.**  You don't need to turn them in.

Now let's look at the frequency-domain data.

**What are the frequencies of the two spikes?**

YOUR ANSWER HERE

**Why are there two spikes even though there is only one sinusoid plotted?**

YOUR ANSWER HERE

**Are the spikes the same for the sine and cosine waveforms?  Why or why not?**

YOUR ANSWER HERE

Move the **tone freq** slider around between (approx) -10Hz and (approx) +10Hz until you get a feel for what you're seeing.

Select the **cosine** waveform and manually enter +10.0Hz for the **tone freq**.  Note what you see.

Now enter -10Hz for the **tone freq**.

**What happened on the plots?  Why?**

YOUR ANSWER HERE

Now select the **sine** waveform and manually enter +10.0Hz for the **tone freq**. Note what you see.

Now enter -10Hz for the tone freq.

**What happened on the plots? Why?**

YOUR ANSWER HERE

Now let's start playing with the sample rate.

Move the **sample rate** slider between (approx) 50Hz and (approx) 100Hz until you get a feel for what you're seeing.

**What, if anything, changes on the left plot?  What, if anything, stays the same?  Why is this the case?**

YOUR ANSWER HERE

**What, if anything, changes on the right plot?  What, if anything, stays the same?  Why is this the case?**

YOUR ANSWER HERE

Choose the **cosine** waveform.  Enter +5.0Hz for the **tone freq** and 20.0Hz for the **sample rate**.  Note what you see.

Now enter +25.0Hz, +45.0Hz, and -15.0Hz into the **tone freq** and note what happens.

**What, if anything, changed on the left plot?  What, if anything, stayed the same?  Why?**

YOUR ANSWER HERE

**What, if anything, changed on the right plot?  What, if anything, stayed the same?  Why?**

YOUR ANSWER HERE

Repeat the last two questions for the **sine** waveform.

**What, if anything, was different about the sine vs the cosine?**

YOUR ANSWER HERE

These equations should look familiar:
\begin{align}
    \cos(2\pi f_o t) &= \frac{e^{j 2\pi f_o t} + e^{-j 2\pi f_o t}}{2} \\
    \sin(2\pi f_o t) &= \frac{e^{j 2\pi f_o t} - e^{-j 2\pi f_o t}}{2j}
\end{align}

**Explain how these equations relate to the exercises above.**

YOUR ANSWER HERE

The above equations are continuous-time because $t$ can take on any value in the range $[-\infty,+\infty]$.

Let's look at what happens when we sample the signal periodically at some sample rate, $f_s$, which corresponds to a sample period $T_s = 1/f_s$.

**If $f_s$ is in Hz, what are the units of $T_s$?  Does this make sense?**

YOUR ANSWER HERE

Convince yourself that if we start sampling at $t = 0$ then the $n^{th}$ sample will occur at $t_n = n T_s$.

Let's focus on sampling a complex exponential with some arbitrary frequency $f_o$:
\begin{align}
    x(t) = e^{j 2\pi f_o t}
\end{align}
The $n^{th}$ sample will be:
\begin{align}
    x[n] &= x(t_n) \\
        &= x(n T_s) \\
        &= e^{j 2\pi f_o n T_s}
\end{align}
where I'm using square brackets to indicate "discrete time" (i.e., sample number instead of seconds).

**MAKE SURE YOU CAN FOLLOW THE ABOVE EQUATIONS BEFORE CONTINUING!**

What happens if we increase the exponential's frequency by the sample rate, $f_s$?  Let find out!
\begin{align}
    e^{j 2\pi (f_o + f_s) n T_s} &= e^{j 2\pi f_o n T_s} \cdot e^{j 2\pi f_s n T_s} \\
        &= e^{j 2\pi f_o n T_s} \cdot e^{j 2\pi n} \\
        &= e^{j 2\pi f_o n T_s} \cdot 1 \\
        &= e^{j 2\pi f_o n T_s} \\
        &= x[n]
\end{align}
Holy smokes - we get the same sample out!  **This is the source of aliasing!**

**Justify the first line of the above proof.**

YOUR ANSWER HERE

**Justify the second line of the above proof.**

YOUR ANSWER HERE

**Justify the third line of the above proof.**

YOUR ANSWER HERE

**Justify the fourth line of the above proof.**

YOUR ANSWER HERE

The last line of the proof follows from the definition of $x[n]$ above.

**Because sines and cosines are built from complex exponentials, what does the above proof imply about sampling them?**

YOUR ANSWER HERE

**What would happen if we changed the exponential's frequency by an integer multiple of the sample rate ($f_o + k f_s$, for integer $k$)?**

YOUR ANSWER HERE

**Does the above proof explain what you saw in any of the steps above?  Which ones?  How?**

YOUR ANSWER HERE

Play around with the plot widget settings.

**Under what circumstance(s) does aliasing occur?  Why?**

YOUR ANSWER HERE

If you made it this far, **congratulations** - I really am impressed!
You're well on your way to understanding many digital signal processing (DSP) concepts...